# Load CSV data

This tutorial provides an example of how to load CSV data from a file into a tf.data.Dataset.

The data used in this tutorial are taken from the Titanic passenger list. The model will predict the likelihood a passenger survived based on characteristics like age, gender, ticket class, and whether the person was traveling alone.

## Setup

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

## Load data

To start, let's look at the top of the CSV file to see how it is formatted.

In [7]:
# !head {train_file_path}

You can load this using pandas, and pass the NumPy arrays to TensorFlow. If you need to scale up to a large set of files, or need a loader that integrates with TensorFlow and tf.data then use the tf.data.experimental.make_csv_dataset function:

The only column you need to identify explicitly is the one with the value that the model is intended to predict.

In [8]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

Now read the CSV data from the file and create a dataset.

(For the full documentation, see tf.data.experimental.make_csv_dataset)

In [9]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [10]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

Each item in the dataset is a batch, represented as a tuple of (many examples, many labels). The data from the examples is organized in column-based tensors (rather than row-based tensors), each with as many elements as the batch size (5 in this case).

It might help to see this yourself.

In [11]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
age                 : [20. 21. 28. 28. 48.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 9.846  7.75  13.    30.696  7.854]
class               : [b'Third' b'Third' b'Second' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'y']


As you can see, the columns in the CSV are named. The dataset constructor will pick these names up automatically. If the file you are working with does not contain the column names in the first line, pass them in a list of strings to the column_names argument in the make_csv_dataset function.

In [12]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'female' b'female' b'female' b'female']
age                 : [28. 30. 28. 30. 42.]
n_siblings_spouses  : [0 3 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 7.75  21.     7.879 12.475 13.   ]
class               : [b'Third' b'Second' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']


This example is going to use all the available columns. If you need to omit some columns from the dataset, create a list of just the columns you plan to use, and pass it into the (optional) select_columns argument of the constructor.

In [13]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 17. 28. 28. 28.]
n_siblings_spouses  : [0 0 1 0 8]
class               : [b'Third' b'Third' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'y' b'n' b'y' b'n']


## Data preprocessing

A CSV file can contain a variety of data types. Typically you want to convert from those mixed types to a fixed length vector before feeding the data into your model.

TensorFlow has a built-in system for describing common input conversions: tf.feature_column, see this tutorial for details.

You can preprocess your data using any tool you like (like nltk or sklearn), and just pass the processed output to TensorFlow.

The primary advantage of doing the preprocessing inside your model is that when you export the model it includes the preprocessing. This way you can pass the raw data directly to your model.

### Continuous data

If your data is already in an appropriate numeric format, you can pack the data into a vector before passing it off to the model:

In [14]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [36. 26. 24. 34. 17.]
n_siblings_spouses  : [1. 0. 0. 0. 0.]
parch               : [1. 0. 0. 1. 0.]
fare                : [24.15   7.896  8.05  23.    12.   ]


In [15]:
example_batch, labels_batch = next(iter(temp_dataset)) 

Here's a simple function that will pack together all the columns:

In [16]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

Apply this to each element of the dataset:

In [17]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 1.     1.     2.    20.575]
 [41.     0.     5.    39.688]
 [25.     1.     1.    30.   ]
 [28.     8.     2.    69.55 ]
 [33.     1.     1.    20.525]]

[1 0 1 0 0]


If you have mixed datatypes you may want to separate out these simple-numeric fields. The tf.feature_column api can handle them, but this incurs some overhead and should be avoided unless really necessary. Switch back to the mixed dataset:

In [18]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'male']
age                 : [38. 19. 23. 28. 28.]
n_siblings_spouses  : [1 0 3 0 0]
parch               : [5 0 2 0 0]
fare                : [ 31.388   7.896 263.      8.05    7.896]
class               : [b'Third' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'C' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'n' b'y' b'y']


In [19]:
example_batch, labels_batch = next(iter(temp_dataset)) 

So define a more general preprocessor that selects a list of numeric features and packs them into a single column:

In [20]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [21]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [22]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'female']
class               : [b'First' b'First' b'First' b'Third' b'Second']
deck                : [b'E' b'C' b'D' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'n' b'y' b'n']
numeric             : [[30.     0.     0.    56.929]
 [52.     0.     0.    30.5  ]
 [19.     0.     2.    26.283]
 [28.     0.     0.     7.25 ]
 [14.     1.     0.    30.071]]


In [23]:
example_batch, labels_batch = next(iter(packed_train_data)) 

### Data Normalization

Continuous data should always be normalized.

In [25]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [26]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [28]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

Now create a numeric column. The tf.feature_columns.numeric_column API accepts a normalizer_fn argument, which will be run on each batch.

Bind the MEAN and STD to the normalizer fn using functools.partial.

In [29]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x00000255106DDEE8>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

When you train the model, include this feature column to select and center this block of numeric data:

In [30]:
example_batch['numeric']

<tf.Tensor: id=550, shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  0.   ,  0.   , 26.55 ],
       [24.   ,  2.   ,  3.   , 18.75 ],
       [28.   ,  0.   ,  0.   ,  7.312],
       [37.   ,  0.   ,  0.   ,  9.587],
       [32.5  ,  1.   ,  0.   , 30.071]], dtype=float32)>

In [31]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479, -0.144],
       [-0.45 ,  1.264,  3.304, -0.286],
       [-0.13 , -0.474, -0.479, -0.496],
       [ 0.589, -0.474, -0.479, -0.454],
       [ 0.229,  0.395, -0.479, -0.079]], dtype=float32)

The mean based normalization used here requires knowing the means of each column ahead of time.

## Categorical data

Some of the columns in the CSV data are categorical columns. That is, the content should be one of a limited set of options.

Use the tf.feature_column API to create a collection with a tf.feature_column.indicator_column for each categorical column.

In [32]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [33]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [34]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [35]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


This will be become part of a data processing input later when you build the model.

## Combined preprocessing layer

Add the two feature column collections and pass them to a tf.keras.layers.DenseFeatures to create an input layer that will extract and preprocess both input types:

In [36]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [37]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     1.     0.     0.     0.     0.     1.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13  -0.474
 -0.479 -0.144  1.     0.   ]


## Build the model

Build a tf.keras.Sequential, starting with the preprocessing_layer.

In [38]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

## Train, evaluate, and predict

Now the model can be instantiated and trained.

In [39]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [40]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 1s 12ms/step - loss: 0.5143 - accuracy: 0.7608
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4223 - accuracy: 0.8309
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4105 - accuracy: 0.8325
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3966 - accuracy: 0.8341
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3817 - accuracy: 0.8389
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3847 - accuracy: 0.8373
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3686 - accuracy: 0.8533
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3647 - accuracy: 0.8453
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3478 - accuracy: 0.8517
Epoch 10/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3508 - accuracy: 0.845

Once the model is trained, you can check its accuracy on the test_data set.

In [41]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 8ms/step - loss: 0.4491 - accuracy: 0.8447


Test Loss 0.4491254732293903, Test Accuracy 0.8446969985961914


Use tf.keras.Model.predict to infer labels on a batch or a dataset of batches.

In [42]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))



Predicted survival: 8.94%  | Actual outcome:  DIED
Predicted survival: 92.36%  | Actual outcome:  DIED
Predicted survival: 93.68%  | Actual outcome:  SURVIVED
Predicted survival: 7.04%  | Actual outcome:  DIED
Predicted survival: 99.94%  | Actual outcome:  DIED
